# PREDICTIVE MODELINGS FOR NEW MEMBERSHIP ACQUISITION

In this file, you will see a new member acquisition model built on 400K Non-member in 2018, and we are predicting whether they became member in 2019. 

And this modeling scripts are built by CAL Team (Patrick Seng, Sam Musch, Pardha, Sameeksha and Shaco) collective efforts. For any issues and doubts, feel free to message us. 

# Preparation

The below chunks are used for importing built-in packages for modeling training. You don't need to change the content. Just run it. These packages are like stored procedures in SQL, written functions.

In [1]:
%matplotlib inline
# import necessary libraries and specify that graphs should be plotted inline. 
# from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import cross_val_score, train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report,auc, accuracy_score, confusion_matrix, roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pydotplus
import seaborn as sns
from sklearn.tree import export_graphviz
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE

ImportError: cannot import name '__check_build' from partially initialized module 'sklearn' (most likely due to a circular import) (c:\users\sande\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\__init__.py)

The below chunk is used for importing the dataset for training.

We basically need to files for model training:
* Behavior related data, contains their engagement scores, past memberships, click through rate etc
* Demographic related data, contains their demographic features including household income, age etc.


In the future, when you need to read data from other files, you just need to change work directory. 
For example, instead of doing `pd.read_csv('Data_new_member_training_final.csv')`, 
you can do 
* `pd.read_csv('..\5 Predictive Scriptis\Data_new_member_training_final.csv')`

Add the relative path in front of the file name.
`..\filename` leads you to the parent directory.

In [ ]:
# set work directory
path = ''
# load the data
past_five = pd.read_csv(path + 'Data_new_member_training_final.csv')
# demographic features
individual = pd.read_csv('individual_info_cleaned.csv')

The below chunk helps you to know the records in the data.
You just need to have `filename.head(N)` to print the most top records in your data.

If you want to see 10 records, just change the `number_of_records_to_print` accordingly.

In [4]:
number_of_records_to_print = 5
past_five.head(number_of_records_to_print)

NameError: name 'past_five' is not defined

In [ ]:
individual.head(number_of_records_to_print)

This chunk below gives you what columns are included in your data file.

Every time you want to see it, just do `file_name.columns`

In [ ]:
past_five.columns

This chunk helps you select needed columns for training. You can put/select columns that existed in the data by printing their columns like we did above, and then put those names in the brackets

**Feature Selection**

We exclude UMAA score since there is only 1 year data. Similarly, we use `click_through_rate` to measure how customer respond to our emails. IN addition, we alsu want to use sub-category engagement scores. 

We didn't use `UMN_MEMBER` score since we think this score doesn't truly reflect members' status. For example, for non-members, they definitely don't have a good UMN MEMBER score, but they are also likely to become a great member. 

In [ ]:
# decide to keep which columns
individual = individual[['ID_DEMO','GENDER','AGE','HOUSEHOLD_INCOME','IN_TC_METRO_AREA','MEMBERSHIP_STATUS_CODE']]

In [5]:
past_five_new = past_five[['ID_DEMO','membership_LastYear','UMN_event','UMN_donor','UMN_volun', 
                           'UMN_inform','UMN_loyalty', 'annual_years',
                           'nonmem_years','membership_TwoYearsAgo', 'general_ctr']]

NameError: name 'past_five' is not defined

# Data Prepration

Create a new feature called `past_member` to indicate whether this person was a member before. `Membership_status_code` indicated their past membership status. 

In [ ]:
# No need to change this part in the future. The objective of this part is to craetea a new column
# For people who were a past member, we use 1. Otherwise, we use 0.
individual.loc[individual['MEMBERSHIP_STATUS_CODE'] == 'P','PAST_MEMBER'] = 1
individual.drop(['MEMBERSHIP_STATUS_CODE'],axis = 1, inplace = True)
individual.PAST_MEMBER.fillna(0, inplace = True)

For those people who are annual member in 2018, we are predict their membership status in 2019. 

Once we have the model, train the model well, we will use it to predict the membership status for 2020

In [ ]:
# This chunk helps us select people who are annual member last year.
# No need to change the this chunk in the future, just run it.
past_five_new = past_five_new[past_five_new.membership_TwoYearsAgo == 'non-member']

This chunk is similar to the `Join` function in SQL, we join two tables together to store information in only one table.

**Parameters**
* New table name is "data", `how = 'xxx'`, can be inner, left or right. 
* `on = 'column name'`,  the column you define as key
* if you have multiple columns, you can do `left_on = ['column1',column2]`,`right_on = ['column1','column2']`

For now, just run it, and no need to change

In [ ]:
data = pd.merge(past_five_new, individual, how = 'inner', on = 'ID_DEMO')

In [ ]:
# There are unique 420749 Non-members in 2018
data.ID_DEMO.nunique()

The below chunk shows the missingness for each column. We want there is no missing values in our data. 

In [6]:
1 - data.count() / len(data.index)

NameError: name 'data' is not defined

There is missing values in HOUSEHOLD_INCOME column. We removed those rows.

In [ ]:
# only keep those records without missing values
data = data[data.HOUSEHOLD_INCOME.notnull()]

# Feature Engineer

In [ ]:
# print head
data.head()

Our model isn't as smart as human. So sometime the model cannot understand all values.

In this table, we need to do several things:
* convert gender information from letters (M,F) to numbers (1,0)
* convert IN_TC_METRO_AREA information from letters (Y,N) to numbers (1,0)

In [ ]:
data.GENDER = np.where(data.GENDER == 'M',1,0)
data.IN_TC_METRO_AREA = np.where(data.IN_TC_METRO_AREA == 'Y',1,0)

In [ ]:
data.head()

In [ ]:
# rename the column names
# just want to keep them as UPPER case
data.rename({'UMN_event':'UMN EVENT', "UMN_member":"UMN MEMBER","UMN_donor":"UMN DONOR","UMN_volun":"UMN VOLUN","UMN_inform":"UMN INFORM",\
                   "UMN_loyalty":"UMN LOYALTY",'annual_years':'ANNUAL YEARS', 'life_years':"LIFE YEARS",\
       'nonmem_years':"NOT MEMBER YEARS", 'life_before_15_flag':"IF LIFE MEMBER BEFORE 2015", 'ctr':"CLICK THROUGH RATE",\
       'events_total':'EVENTS ATTENDED IN TOTAL','general_ctr':"CLICK THROUGH RATE"}, inplace = True, axis =1)

# Split Data

We want to create another `table` named as `data_final` in case we do something wrong. So we create another table as our backup. When we want to go back to see our orignal data, we can just print `data`.

In [7]:
data_training = data.copy()

NameError: name 'data' is not defined

**How we set the prediction flag**

If the membership status in 2019 is a member, set the value to 1, else 0.

* Membership status in 2019 is lifetime, then 1
* Membership status in 2019 is annual member, then 1
* Membership status in 2019 is not a member, then 0

In [ ]:
# create predict/target variable
data_training['membership_LastYear'] = np.where(data_training.membership_LastYear == 'non-member',0,1)
# rename the column
data_training.rename(columns = {'membership_LastYear':'TARGET'}, inplace = True)
# drop the column 'membership 2018'
data_training.drop(['membership_TwoYearsAgo'],inplace = True, axis = 1)

In [ ]:
data_training.head()

### Resample The Data

There is a **big challenge** for predictive modeling which is called imbalanced data. When there are overwhelming values than others, the model cannot perform very well. 

For example, there are 100 annual members, 95 of them will leave. A bad model can predict all of them that they won't renew their membership. However, we lose the opportunity to identify the remaining 5 who will ocnvert to a lifetime member while the model predicts 95% correct. 

So we need to deal with the resample technique to solve the imbalance issue.

It's hard to explain the detail numbers in below chunks. They are selected based on testing. I would recommend for the future analysis, find someone who knows more about the resampling techniques to try a better parameter training.

The below chunk is what we found work best for the current data. While the data growing, the parameters might change in the future.

In [ ]:
# only 0.8% of non-member convert to members in 2019
# see the distribution of target variable
# table_name. column_name . value_counts() and 
# then you can know how many people lables 1 and how many labels 0
data_training.TARGET.value_counts()

We noticed that there are way more "not member" in our data. If the model simply predict all of them as not member, the model can still achieve a relative high accraucy, which is not we desired. 

In order to solve this imbalanced data issues, we decided to use resample techniques to simulate some new data and reduce the imbalance. 

In [ ]:
more = data_training[data_training.TARGET == 0] # 449671 records
less = data_training[data_training.TARGET == 1] # 3009 records

df_unsampled_less = resample(less, replace = True, n_samples = 9000, random_state = 42)
df_unsampled = pd.concat([more,df_unsampled_less])

In [ ]:
# have the training and testing data readt
y = df_unsampled['TARGET']
x = df_unsampled.drop(['ID_DEMO','TARGET'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.33, random_state = 42, stratify = y)

sm = SMOTE(random_state = 42, sampling_strategy= 0.50)
X_train, y_train = sm.fit_sample(X_train, y_train.ravel())

# Random Forest for Non-Member

**Random Forest Model**

Below is a model called random forest.

We train the model based on previous years data, and predict membership status next year. The below chunks trains the model.

In [ ]:
# hyper parameters to tune
# n_estimators = [100,300,500]
# max_depth = [4,6,8]
# min_samples_split = [2,4]
# hyper = dict(n_estimators = n_estimators, max_depth = max_depth, 
#              min_samples_split = min_samples_split)
# rf = RandomizedSearchCV(RandomForestClassifier(),hyper,cv = 3)

# setup the model
rf = RandomForestClassifier(n_estimators= 100, max_depth = 8,min_samples_split=2)
# fit the model
rf.fit(X_train,y_train)
# predict the result
pred_rf_no = rf.predict(X_test)
pred_rf_no_all = rf.predict(x)

In [ ]:
# Get the model performance
# the result will be printed out by running the below code
print(classification_report(y_test, pred_rf_no))

# Get all measurement
columns_name = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
accuracy = accuracy_score(y_test, pred_rf_no)
print('Accuracy of Decision Tree is {0}'.format(round(accuracy,4)))

In [ ]:
# output probability for new member acquisition
predictions_rf_no = [round(1-i[0],3) for i in rf.predict_proba(x)]
output_rf_no = pd.DataFrame({'ID_DEMO':df_unsampled['ID_DEMO'].values,
                             'Prob_NotMember':predictions_rf_no,
                            'Member_Label':pred_rf_no_all})
# drop duplicates
output_rf_no = output_rf_no.drop_duplicates()

In [ ]:
# feature importance
feature_importance = pd.DataFrame(rf.feature_importances_,
                                 index = X_train.columns,
                                 columns = ['importance']).sort_values('importance', 
                                                                       ascending = True)
feature_importance.sort_values(by = 'importance', ascending = False).head()
# output the feature importance for new member acquisition
# feature_importance.sort_values(by = 'importance', ascending = False).to_csv('Feature_importance_New_member.csv')

In [ ]:
plt.figure(figsize=(10,8))
plt.barh(y = feature_importance.index, width = feature_importance.importance)
plt.xlabel('Relative Feature Importance', fontsize =  16)
plt.xticks(size = 14)
plt.yticks(size = 14)
plt.show()

# New Member Acquistion Prediction

In [ ]:
# read data for prediction
data_next = pd.read_csv(path + 'Data_new_member_future_prediction_final.csv')
individual = pd.read_csv(path + 'individual_info_cleaned.csv')

In [ ]:
# exclude people who are actually current members
individual = individual[individual.MEMBERSHIP_STATUS_CODE != 'C']

In [ ]:
# select useful features from individual
individual = individual[['ID_DEMO','GENDER','AGE',
        'IN_TC_METRO_AREA', 'HOUSEHOLD_INCOME', 'MEMBERSHIP_STATUS_CODE',]]

In [ ]:
# No need to change this part in the future. The objective of this part is to craetea a new column
# For people who were a past member, we use 1. Otherwise, we use 0.
individual.loc[individual['MEMBERSHIP_STATUS_CODE'] == 'P','PAST_MEMBER'] = 1
individual.drop(['MEMBERSHIP_STATUS_CODE'],axis = 1, inplace = True)
individual.PAST_MEMBER.fillna(0, inplace = True)

In [ ]:
# merge demographic information
data_prediction = pd.merge(data_next,individual, how = 'inner', on = 'ID_DEMO')

In [ ]:
data_prediction.head()

In [ ]:
# check missing values
data_prediction.isnull().sum()

In [ ]:
# only keep those records without missing values
data_prediction = data_prediction[data_prediction.HOUSEHOLD_INCOME.notnull()]

In [ ]:
# people who are annual member last year
data_prediction = data_prediction[data_prediction.membership_LastYear == 'non-member']

In [ ]:
# rename the column names
# just want to keep them as UPPER case
data_prediction.rename({'UMN_event':'UMN EVENT', "UMN_member":"UMN MEMBER","UMN_donor":"UMN DONOR","UMN_volun":"UMN VOLUN","UMN_inform":"UMN INFORM",\
                   "UMN_loyalty":"UMN LOYALTY",'annual_years':'ANNUAL YEARS', 'life_years':"LIFE YEARS",\
       'nonmem_years':"NOT MEMBER YEARS", 'life_before_15_flag':"IF LIFE MEMBER BEFORE 2015", 'ctr':"CLICK THROUGH RATE",\
       'events_total':'EVENTS ATTENDED IN TOTAL','general_ctr':"CLICK THROUGH RATE"}, inplace = True, axis =1)

In [ ]:
# similarly, we need to change the data values to the type that our model can recognize 
data_prediction.GENDER = np.where(data_prediction.GENDER == 'M',1,0)
data_prediction.IN_TC_METRO_AREA = np.where(data_prediction.IN_TC_METRO_AREA == 'Y',1,0)

In [ ]:
x = data_prediction[['UMN EVENT', 'UMN DONOR', 'UMN VOLUN', 'UMN INFORM', 'UMN LOYALTY',
       'ANNUAL YEARS', 'NOT MEMBER YEARS', 'CLICK THROUGH RATE', 'GENDER',
       'AGE', 'HOUSEHOLD_INCOME', 'IN_TC_METRO_AREA', 'PAST_MEMBER']]

In [ ]:
x.head()

In [ ]:
# previous model is called 'rf', then use rf to train the new model
pred_rf_new = rf.predict(x)

In [ ]:
# Output prediction result.
# the CSV file will generated in the end of this file, along with the probability of churn analysis.
predictions_rf_new = [1 - round(i[0],3) for i in rf.predict_proba(x)]
output_rf_new = pd.DataFrame({'ID_DEMO':data_prediction['ID_DEMO'].values,
                               'Probability_NewMember':predictions_rf_new})
# drop duplicates
output_rf_new = output_rf_new.drop_duplicates()
output_rf_new.head()

# Export Probability

In [ ]:
individual = pd.read_csv(path + 'individual_info_cleaned.csv')
output_rf_new = pd.merge(output_rf_new,individual, how = 'inner', on = 'ID_DEMO')

In [ ]:
output_rf_new.head()

In [ ]:
output_rf_new.shape

In [ ]:
output_rf_new.to_csv('Prediction_NewMember_Acquisition.csv', index = False)